# 1. Importing Libraries

In [23]:
import numpy as np
import pandas as pd

from sklearn.model_selection import (
    train_test_split,
    cross_val_score
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import (
    r2_score,
    make_scorer
)

import joblib

# 2. Getting the Data

In [4]:
df = pd.read_csv(
    "./data/car-details.csv", 
    usecols=[
        "company",
        "year",
        "owner",
        "fuel",
        "km_driven",
        "mileage_mpg",
        "engine_cc",
        "seats",
        "selling_price" 
    ]
)

In [5]:
df.head()

,company,year,owner,fuel,km_driven,mileage_mpg,engine_cc,seats,selling_price
0,Maruti,2014,First,Diesel,145500,55.00,1248.0,5.0,450000
1,Skoda,2014,Second,Diesel,120000,49.70,1498.0,5.0,370000
2,Honda,2006,Third,Petrol,140000,41.60,1497.0,5.0,158000
3,Hyundai,2010,First,Diesel,127000,54.06,1396.0,5.0,225000
4,Maruti,2007,First,Petrol,120000,37.84,1298.0,5.0,130000


# 3. Analyzing the Data

In [9]:
df.shape

(6926, 9)

In [6]:
df.dtypes

company           object
year               int64
owner             object
fuel              object
km_driven          int64
mileage_mpg      float64
engine_cc        float64
seats            float64
selling_price      int64
dtype: object

In [7]:
def missing_info(temp_df):
    na_cols = [col for col in temp_df.columns if temp_df[col].isna().any()]
    na_counts = [temp_df[col].isna().sum() for col in na_cols]
    na_pcts = [temp_df[col].isna().mean() for col in na_cols]

    return (
        pd.DataFrame(data = {
            "column" : na_cols,
            "count" : na_counts,
            "pct" : na_pcts
        }).sort_values(by="count", ascending=False).set_index("column")
    )

In [8]:
missing_info(df)

,count,pct
column,,
mileage_mpg,208,0.030032
engine_cc,208,0.030032
seats,208,0.030032


# 4. Splitting the Data

In [11]:
X = df.drop(columns="selling_price")
y = df.selling_price.copy()

X_train, X_test , y_train, y_test = train_test_split(
    X,y, test_size = 0.2, shuffle=True, random_state=42
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5540, 8) (5540,)
(1386, 8) (1386,)


# 5. Preprocessing the Data

In [12]:
X_train.head()

,company,year,owner,fuel,km_driven,mileage_mpg,engine_cc,seats
4855,Ford,2018,First,Diesel,25000,54.06,1498.0,5.0
3143,Maruti,2010,Second,Petrol,170000,NaN,NaN,NaN
4275,Hyundai,2011,Second,Diesel,75500,54.06,1396.0,5.0
2288,Maruti,2009,Second,Diesel,138000,45.34,1248.0,5.0
3914,Hyundai,2003,Second,Diesel,200000,30.78,1493.0,5.0


In [13]:
nums_cols = X_train.select_dtypes(exclude="O").columns.to_list()
cat_cols = np.setdiff1d(X_train.columns,nums_cols).tolist()

In [14]:
num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scalar", StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers= [
    ("num", num_pipe, nums_cols),
    ("cat", cat_pipe, cat_cols)
])

# 6. Training the Model

In [15]:
reg = RandomForestRegressor(random_state=42)

model = Pipeline(steps=[
    ("pre", preprocessor),
    ("reg", reg)
])

model.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scalar',
                                                                   StandardScaler())]),
                                                  ['year', 'km_driven',
                                                   'mileage_mpg', 'engine_cc',
                                                   'seats']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['company', 'fuel',
                                                   'owner'])])),
                ('reg', RandomForestRegressor(random_state=42))])

# 7. Evaluate the Model

In [16]:
y_test_pred = model.predict(X_test)

r2_score(y_test, y_test_pred)

0.8220883761156546

### 7.1 Adjusted R2

In [17]:
def adj_r2(y_test, y_pred, **kwargs):
    n, p = kwargs["data"].shape
    r2 = r2_score(y_test, y_pred)
    return 1 - ((1-r2) * (n - 1) / (n-p-1))
 

In [18]:
adj_r2(y_test, y_test_pred, data = X_train)


0.8218310459780529

In [20]:
adj_r2_score = make_scorer(
    adj_r2,
    response_method = "predict",
    greater_is_better = True,
    data= X_train
)

### 7.2 Cross-velidation Score

In [21]:
cv_score = cross_val_score(
    model,
    X_train,
    y_train,
    scoring=adj_r2_score,
    cv=3,
    n_jobs=-1
)

np.mean(cv_score)

0.847264077129562

# 8. Save the Model

In [24]:
joblib.dump(model, "model.joblib")

['model.joblib']

# 9. Load the Model

In [25]:
saved_model = joblib.load("model.joblib")
saved_model

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scalar',
                                                                   StandardScaler())]),
                                                  ['year', 'km_driven',
                                                   'mileage_mpg', 'engine_cc',
                                                   'seats']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['company', 'fuel',
                                                   'owner'])])),
                ('reg', RandomForestRegressor(random_state=42))])

# 10. Making Predictions

In [26]:
X_test.join(y_test).head(3)

,company,year,owner,fuel,km_driven,mileage_mpg,engine_cc,seats,selling_price
6878,Toyota,2009,First,Diesel,250000,30.08,2494.0,7.0,550000
3660,Mahindra,2015,First,Diesel,39000,37.60,2179.0,7.0,1225000
5604,Ford,2018,First,Diesel,90000,54.06,1498.0,5.0,850000


In [27]:
new_df = pd.DataFrame(data= dict(
    company=["Ford"],
   	year=[2018],
   	owner=["First"],
   	fuel=["Diesel"],
   	km_driven=[90000],
   	mileage_mpg=[54.06],
   	engine_cc=[1498.0],
   	seats=[5.0]
))

new_df

,company,year,owner,fuel,km_driven,mileage_mpg,engine_cc,seats
0,Ford,2018,First,Diesel,90000,54.06,1498.0,5.0


In [28]:
predicted_value = saved_model.predict(new_df)[0]
predicted_value

798095.0